In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


In [ ]:
#载入数据集
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

#定义批次大小 每次训练放入图片数量
batch_size=100

#计算一共多少个批次 (图片数量除以（//)批次大小）
n_batch=mnist.train.num_examples // batch_size


#参数概要
def variable_summaries(var):
    with tf.name_scope('summaires'):
        mean=tf.reduce_mean(var)
        #平均值
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        
#初始化权值
with tf.name_scope('weight'):
    def weight_variable(shape):
        initial=tf.truncated_normal(shape,stddev=0.1)
        return tf.Variable(initial)
#初始化偏值
with tf.name_scope('bias'):
    def bias_variable(shape):
        initial=tf.constant(0.1,shape=shape)
        return tf.Variable(initial)
#卷积层
with tf.name_scope('Conv_layer'):
    def conv2d(x,W):
    #x,input tensor of shape[batch, in_height,in_width,inchannels]
    #W filter/kernel tensor of shape[filter_height, filter_width,in_channels, out_channels]
    #strides[0]=strides[3]=1.strides[1]步长定义，第一个第四个值都是1，第二个X步长，第三个Y步长
    #padding:A string from: "SAME","VALID"
        return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
#池化层
with tf.name_scope('pool'):
    def max_pool_2x2(x):
    #ksize[1,x,y,1]
        return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
#定义place
with tf.name_scope('input'):
    x=tf.placeholder(tf.float32,[None,784],name='input_x')#28*28
    y=tf.placeholder(tf.float32,[None,10],name='input_y')

    with tf.name_scope('input_image'):
# 改编X的格式转为4D向量 [batch,in_height,in_width,inchannels]
        x_image=tf.reshape(x,[-1,28,28,1],name='image_input')

with tf.name_scope('Conv1'):
#初始化第一个卷积层权值和偏值
    with tf.name_scope('W_c1'):
        W_conv1=weight_variable([5,5,1,32])#5*5采样窗口，32个卷积核从一个平面抽取特征
        
    with tf.name_scope('b_conv1'):
        b_conv1=bias_variable([32])#每个卷积核一个偏置值
        

#把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    with tf.name_scope('Conv2d_1'):
        conv2d_1=conv2d(x_image,W_conv1)+b_conv1
    with tf.name_scope('relu'):   
        h_conv1=tf.nn.relu(conv2d_1)
    with tf.name_scope('h_pool1'):
        h_pool1=max_pool_2x2(h_conv1)

#初始化第二个卷积层权值和偏值
with tf.name_scope('Conv2'):
    with tf.name_scope('W_conv2'):
        W_conv2=weight_variable([5,5,32,64])#5*5采样窗口，64个卷积32个平面抽取特征
        
    with tf.name_scope('b_conv2'):
        b_conv2=bias_variable([64])#每个卷积核一个偏置值
        

#把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    with tf.name_scope('conv2d_2'):
        conv2d_2=conv2d(h_pool1,W_conv2)+b_conv2
    with tf.name_scope('relu'):   
        h_conv2=tf.nn.relu(conv2d_2) 
    with tf.name_scope('h_pool2'):
        h_pool2=max_pool_2x2(h_conv2)#进行Max——pooling

#28*28图片第一次卷积后还是28*28， 第一次池化后14*14， 第二次卷积后还是14*14，第二次池化后变为7*7.最后成为64张7*7的图片

#初始化第一个全连接层的权值
with tf.name_scope('fc1'):
    with tf.name_scope('W_fc1'):
        W_fc1=weight_variable([7*7*64,1024])#上一场有7*7*64个神经元，全连接层有1024个神经元
        
    with tf.name_scope('b_fc1'):
        b_fc1=bias_variable([1024])#1024 nodes
        

#池化层2的输出扁平化为1维
    with tf.name_scope('h_pool2_flat2'):
        h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64],name='h_pool2_flat')

#求出第一个全连接层的输出
    with tf.name_scope('wx_plus_b1'):
        wx_plus_b1=tf.matmul(h_pool2_flat,W_fc1)+b_fc1
    with tf.name_scope('relu'):
        h_fc1=tf.nn.relu(wx_plus_b1)

#keep_prob用来表示神经元输出
    with tf.name_scope('keep_prob'):
        keep_prob=tf.placeholder(tf.float32,name='keep_probe')
    with tf.name_scope('h_fc1_drop'):
        h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob,name='h_fc1_drop')


#求出第2个全连接层的输出
with tf.name_scope('fc2'):
    with tf.name_scope('W_fc2'):
        W_fc2=weight_variable([1024,10])
    with tf.name_scope('b_fc2'):
        b_fc2=bias_variable([10])
#计算输出
    with tf.name_scope('wx_plus_b2'):
        wx_plus_b2=tf.matmul(h_fc1_drop,W_fc2)+b_fc2       
    with tf.name_scope('prediction'):  
        prediction =tf.nn.softmax(wx_plus_b2)
with tf.name_scope('loss'):
#定义交叉墒代价函数
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction),name='loss')
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):

#使用adamoptimizer优化
    train_step=tf.train.AdamOptimizer(1e-4).minimize(loss)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
#结果存放在布尔型列表中  定义准确率 比较两个agrmax大小，相同就是TRUE 不同就是FALSE。 argmax求最大值所在位置
        correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))

    with tf.name_scope('accuracy'):             
#求准确率 (tf.cast()将布尔型数据转化成数值型 然后计算准确率
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

#合并所有summaries
merged=tf.summary.merge_all() 


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer=tf.summary.FileWriter('logs/tensorboard_test/12/train',sess.graph)
    test_writer=tf.summary.FileWriter('logs/tensorboard_test/12/test',sess.graph)
    
    for i in range(8001):
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.5})
        #记录计算参数 
        summary=sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        train_writer.add_summary(summary,i)
        #记录测试计算参数   
        batch_xs,batch_ys=mnist.test.next_batch(batch_size)
        summary=sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        test_writer.add_summary(summary,i)
        if i%100 == 0:
            acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
            print("Iter"+str(i)+"Testing Accuracy="+str(acc))

    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter0Testing Accuracy=0.1144
Iter100Testing Accuracy=0.599
Iter200Testing Accuracy=0.8141
Iter300Testing Accuracy=0.8359
Iter400Testing Accuracy=0.9259
Iter500Testing Accuracy=0.9417
Iter600Testing Accuracy=0.9474
Iter700Testing Accuracy=0.954
Iter800Testing Accuracy=0.959
Iter900Testing Accuracy=0.9603
Iter1000Testing Accuracy=0.9622
Iter1100Testing Accuracy=0.965
Iter1200Testing Accuracy=0.9655
Iter1300Testing Accuracy=0.9686
Iter1400Testing Accuracy=0.9699
Iter1500Testing Accuracy=0.9681
Iter1600Testing Accuracy=0.9716
Iter1700Testing Accuracy=0.9708
Iter1800Testing Accuracy=0.9732
Iter1900Testing Accuracy=0.9741
Iter2000Testing Accuracy=0.9746
Iter2100Testing Accuracy=0.9751
Iter2200Testing Accuracy=0.9755
Iter2300Testing Accuracy=0.977
Iter2400Testing Accuracy=0.978
Iter2500Testing Accuracy